# 1. Naver API를 이용한 블로그 포스트들의 URL 받아오기


- 현재로서는 가져올 수 있는 블로그 포스트 링크의 개수가 최대 1100개로 제한되어 있는 것으로 보임.<br/>(이게 걸림돌로 작용될 수 있을듯..)
- API를 이용해서 데이터를 호출할 수 있는 횟수가 하루에 2만 5천번으로 제한되어 있음.
- 현재 이태우의 API 키와 비밀번호를 이용하여 코드가 작성되어 있음 (API_PARAMETER)
- 현재는 많은 링크를 가져올 필요가 없으므로, ### 테스트를 위해 넣은 부분 ### 코드블록을 이용하여 API 호출 횟수를 제한함
<br/><br/>

In [25]:
# <Code Reference> 
# - (Github, xotrs/naver-blog-crawler) https://github.com/xotrs/naver-blog-crawler/blob/master/NaverBlogCrawler.py
# - (검색 API) https://developers.naver.com/docs/search/blog/

import os
import sys
import urllib.request
import json
import math

API_PARAMETER = {
    "client_id": "E5cL2__bFGNST8f4eknt",
    "client_secret": "kcwtUqHT98"
}

QUERY_PARAMETER = {
    "query": "네이버 클로바",
    "display": 100,
    "sort_type": "sim"
}

def naver_api_blog_search(api_parameter, search_word, 
                          display_count=10, start=1, sort_type='sim'):
    result = None
    
    client_id = api_parameter['client_id']
    client_secret = api_parameter['client_secret']
    
    encText = urllib.parse.quote("{}".format(search_word))
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&display=" + str(
            display_count) + "&start=" + str(start) + "&sort=" + sort_type  # json 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과 예시
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        result = response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode) 
    return result

def get_search_result_page_count(api_parameter, query_word, display_count):
    search_result_page_count = 0
    
    decoded_response_body = naver_api_blog_search(api_parameter, query_word)
    if decoded_response_body:
        json_response_body = json.loads(decoded_response_body)
        
        total_posts = json_response_body['total']
        if total_posts != 0:
            search_result_page_count = math.ceil(total_posts / display_count)
    return search_result_page_count
    
def get_urls(query_word, display_count, sort_type, search_result_page_count):
    url_list = []
    for i in range(search_result_page_count):
        start = display_count*i + 1
        if start > 1000:
            start = 1000
            
        decoded_response_body = naver_api_blog_search(api_parameter, query_word, display_count, start, sort_type)
        if decoded_response_body:
            json_response_body = json.loads(decoded_response_body)
            
            for dp in json_response_body['items']:
                url_list.append(dp['link'].replace("amp;", ""))
        
        ### 테스트를 위해 넣은 부분 ###
        #if i == 1:
        #    break
        #########################
        
        if start == 1000:
            break
    return url_list


if __name__ == '__main__':
    api_parameter = API_PARAMETER 
    query = QUERY_PARAMETER['query']
    display = QUERY_PARAMETER['display']
    sort_type = QUERY_PARAMETER['sort_type']
    
    p_count = get_search_result_page_count(api_parameter, query, display)
    urls = get_urls(query, display, sort_type, p_count)

In [26]:
urls

['https://blog.naver.com/snowrun?Redirect=Log&logNo=221508004958',
 'https://blog.naver.com/sky121400?Redirect=Log&logNo=221493275584',
 'https://blog.naver.com/danjuck?Redirect=Log&logNo=221502038209',
 'https://blog.naver.com/ralrapo?Redirect=Log&logNo=221418872833',
 'https://blog.naver.com/yarosiku?Redirect=Log&logNo=221410642877',
 'https://blog.naver.com/ukb0914?Redirect=Log&logNo=221355525241',
 'https://blog.naver.com/aimei96?Redirect=Log&logNo=221288615932',
 'https://blog.naver.com/riniwood?Redirect=Log&logNo=221507019476',
 'https://blog.naver.com/three486?Redirect=Log&logNo=221492447164',
 'https://blog.naver.com/clamp0112?Redirect=Log&logNo=221507783401',
 'https://blog.naver.com/iffu?Redirect=Log&logNo=221015291581',
 'https://blog.naver.com/lopil22?Redirect=Log&logNo=221412000291',
 'https://blog.naver.com/bless_yuna?Redirect=Log&logNo=221416528704',
 'https://blog.naver.com/soeun3093?Redirect=Log&logNo=221351131297',
 'https://blog.naver.com/ddiamosho?Redirect=Log&logNo

In [27]:
len(urls)

1100

<br/><br/>
# 2. 받아온 URL들을 이용하여 원하는 정보를 scraping

In [52]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', attrs={'id': re.compile('^post-view')}):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', 'view'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts
        
if __name__ == '__main__':
    
    
    for url in urls:    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)

        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)

        print('url: ', url)
        print('real_url: ', real_url)
        print()
        print('post title: ', title)
        print()
        print('publish date time: ', publish_datetime)
        print()
        print("texts: ", texts)
        print("\n-----------------------------------\n")

        time.sleep(1)

url:  https://blog.naver.com/snowrun?Redirect=Log&logNo=221508004958
real_url:  http://blog.naver.com/PostView.nhn?blogId=snowrun&logNo=221508004958&from=search&redirect=Log&widgetTypeCall=true&directAccess=false

post title:  네이버 클로바 AI 스피커 샀어요 ! 솔직후기

publish date time:  2019-04-08 18:16:46 17시간 전

texts:  네이버 클로바 AI 스피커 샀어요 ! 솔직후기
네이버 클로바 AI 스피커 샀어요 !
브라운
과 함께하는 일상....
솔직후기..
.
♥
안녕하세요 라인프렌즈를 좋아하는 핸잉이입니당
 ♥
_
♥
제가 지난달에 지름신이 왔었다고 했었는데요 !
휴대폰 소독 스프레이(?), 부라더 라벨기에 이어 AI 스피커 네이버 클로바 까지 3일만에 팡팡 질렀답니당ㅋㅋㅋ
네이버클로바를 처음 보고 라인프렌즈라 갖고싶다는 생각이 있었지만
과연 있으면 자주 쓸까..? 하는 생각에 미뤄왔었는데요.
미루다가 또 이렇게 사게됐네욤 ~~
네이버 클로바 브라운과 함께한 시간은 한 2~3주정도??
그동안 쓰면서 느꼈던 솔직한 느낌!! 을 후기로 담아보도록 하겠습니다 *_*
네이버 클로바는 기본 사이즈의 네이버 클로바 , 미니언즈 모양 , 기본사이즈랑 똑같은 디자인인데 좀 더 작은 네이버 클로바 미니가 있어요!
예전에 도라에몽 클로바도 있었는데 다 팔렸나봐욤.. 그런데 이번에 네이버 클로바 럭키위크 이벤트에 도라에몽 클로바가 있더라구요!!!
와 당첨되고 싶다ㅠㅠㅠ(이미 클로바 있으면서..하긴 이벤트에 참여하려면 있어야 하겠구낭)
저는 기본사이즈의 네이버 클로바 브라운으로 선택했어요 !!!
*
네이버클로바가 이름이 네이버 클로바인 이유를 물어보았어요.
나: 넌 왜 이름이 클로바니?
클로바: 사람들이 제 이름을 들으면 네잎클로버가 생각난대요. 행운의 의미니까

KeyboardInterrupt: 

<br/><br/>
# 3. 받아온 URL들을 이용하여 원하는 정보를 scraping 하고 CSV로 저장

In [70]:
# <Code Reference> 
# - (zilu11의 블로그 포스트) https://blog.naver.com/zilu11/221255183159

import requests
from bs4 import BeautifulSoup

import time
import datetime
import re


def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url 

def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            # e.g. 2019. 04. 01. 15:49 4시간 전
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', attrs={'id': re.compile('^post-view')}):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div'):
            #for div in soup.find('div', attrs={'id': 'postViewArea'}).findAll('div', 'view'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

if __name__ == '__main__':
    title_list = []
    publish_datetime_list = []
    text_list = []
    
    for i, url in enumerate(urls):    

        # 실제 데이터가 들어있는 url 추출
        real_url = get_realdata_url_naverblog(url)
        r = requests.get(real_url)
        soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

        # 블로그 포스트 제목 추출
        title = get_title_naverblog(soup)
        title_list.append(title)
        
        # 블로그 작성일 추출
        publish_datetime = det_publish_datetime_naverblog(soup)
        publish_datetime_list.append(publish_datetime)
        
        # 블로그 텍스트 추출
        texts = get_texts_naverblog(soup)
        text_list.append(texts)
        
        print("{}.".format(i), "finished")
        time.sleep(1)

0. finished


KeyboardInterrupt: 

In [23]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 'publish_datetime': publish_datetime_list, 'text': text_list, 'url': urls})

In [25]:
df = df[['title', 'publish_datetime', 'text', 'url']]

In [26]:
df

,title,publish_datetime,text,url
0,애기들이 좋아하는 ai스피커 네이버클로바,2019-03-20 23:01:00,애기들이 좋아하는 ai스피커 네이버클로바\n작년부터 tv에 #인공지능 스피커가 \n...,https://blog.naver.com/sky121400?Redirect=Log&...
1,네이버 클로바 스피커 1년 사용 후기,2019-03-31 19:35:00,네이버 클로바 스피커 1년 사용 후기\n네이버 클로바 1년 사용 후기를 몇 자 적어...,https://blog.naver.com/danjuck?Redirect=Log&lo...
2,네이버 클로바 사용법 맘스다이어리와 함께 육아정보를,2018-12-14 12:10:00,어제는 눈이 갑자기 많이 내려서 1주일에 한번 가는 문센을 가야하나 말아야하나고민이...,https://blog.naver.com/ralrapo?Redirect=Log&lo...
3,네이버 클로바. 나의 두 번째 Ai 스피커,2018-12-02 14:04:00,안녕하세요. 제르디난 입니다.\n올해 초 카카오 미니를 사용해보고 약간의 실망스러움...,https://blog.naver.com/yarosiku?Redirect=Log&l...
4,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!,2018-09-09 18:18:00,네이버 클로바 스피커 브라운 미니언 : 오히려 부모님 선물로 추천드려요!\n요새 정...,https://blog.naver.com/ukb0914?Redirect=Log&lo...
5,우리집AI는 네이버 클로바 미니언즈당 ★,2018-05-31 17:00:00,네이버 클로바 미니언즈로 메이네 집이 스마트해졌습니다! ㅋㅋ 저희가 쓰는 인터넷이 ...,https://blog.naver.com/aimei96?Redirect=Log&lo...
6,네이버 클로바 스피커 사용해보기,2019-03-19 22:57:00,네이버 클로바 스피커 사용해보기\n저는 유행에 둔감한 사람인 것 같습니다.\n아마도...,https://blog.naver.com/three486?Redirect=Log&l...
7,"네이버 클로바, 인공지능 음성인식 비서 앱 만나보니",2017-05-27 08:50:00,"최근 구글 어시스턴트, 아마존 알렉사, 애플 시리, 삼성전자 빅스비 등 인공지능 기...",https://blog.naver.com/iffu?Redirect=Log&logNo...
8,인공지능스피커 LG 유플러스 네이버 클로바 미니언즈 사용기,2018-12-04 11:39:00,꼬물 꼬물 귀요미 미니언즈가LG유플러스 네이버 클로바 스페셜에디션으로나왔어요~♥보기...,https://blog.naver.com/lopil22?Redirect=Log&lo...
9,인공지능스피커 네이버클로바 설치/ 맘스다이어리 연동,2018-12-10 22:53:00,인공지능스피커 네이버 클로바맘스다이어리 천일스토리로 육아정보 들어요\n육아하는 엄마...,https://blog.naver.com/bless_yuna?Redirect=Log...


In [27]:
df.to_csv('./naver_clova.csv')

<br/><br/><br/>
# 예외> 본문 데이터가 중복돼서 가져와지는 것 확인하기

In [74]:
url = "https://blog.naver.com/mchan21/221249916216"
#url = "http://blog.naver.com/PostView.nhn?blogId=yukihana96&logNo=221389690746&from=search&redirect=Log&widgetTypeCall=true&directAccess=false"
real_url = get_realdata_url_naverblog(url)
r = requests.get(real_url)
soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

In [75]:
real_url

'http://blog.naver.com/PostView.nhn?blogId=mchan21&logNo=221249916216&redirect=Dlog&widgetTypeCall=true&directAccess=false'

In [76]:
# 블로그 텍스트 추출
texts = get_texts_naverblog(soup)
print(texts)


 이건 사야해! 네이버 클로바 미니언즈 스피커 개봉기슈퍼배드의 귀여운 악당! 색다른 언어와 아이같은 행동이 특징인 귀여운 미니언즈를 모르는 분들이 없을겁니다.노란색에 바지를 입고 하나 또는 2개의 눈을 가졌으며 고글을 하고 있는 미니언즈가 네이버 클로바 프렌즈와 헙업을 통해 인공지능 블루투스 스피커로 출시되었습니다.저는 평소 네이버 뮤직을 사용해서 네이버에서 선보이는 인공지능 스피커 클로바 제품에 굉장히 관심이 많았는데요.이번에 새롭게 출시된 클로바 프렌즈 미니언즈 에디션의 경우 정기결제 이용자에게 권장소비자가 134000원 제품을 59% 할인으로 55000원에 구입할 수 있어서 바로 주문했습니다.참고로 신규 가입자에게는 48% 할인가 94900원(무제한듣기 6개월 약정 제공), 선물시 45% 할인가 87000원(무제한듣기 3개월 쿠폰 포함)으로 구입이 가능합니다.​ 기존 클로바 프렌즈의 경우 라인의 인기 캐릭터인 브라운과 샐리를 형상화한 디자인으로 출시 당시 이틀만에 1만대가 모두 소진될 만큼 큰 인기를 끌었습니다.​ 저도 네이버의 AI 스피커 프렌즈를 사용하고 있는데요. 평소 좋아하는 캐릭터 브라운을 형상화한 제품으로 저는 매일 아침 라인 프렌즈를 통해 일어나고, 출근 전 날씨정보, 음악감상을 하다가 퇴근 후에는 일정 관리나 브리핑, 스포츠 경기결과 등을 물어보고 자장가를 들으며 잠을 자는 등 하루를 함께하고 있습니다.패키지 박스는 제품 디자인을 제외하면 거의 비슷한 느낌을 받을 수 있습니다.​ 패키지 박스를 열면 귀여운 네이버 클로바 미니언즈 본체를 확인할 수 있습니다. 그 밑으로 구성품들이 빼곡히 들어있습니다.​ 구성품은 심플한데요. 클로바 프렌즈 미니언즈 본체와 충전 어답터, 충전 케이블, 퀵 스타트 가이드 및 품질보증서, 발화문 가이드, 표정을 꾸밀 수 있는 미니언즈 페이스 스티커 등으로 구성되어 있습니다.​ 사용설명서에는 제품의 제원부터 특징, 연결방법, 사용방법, 사용팁 등 상세하게 설명되어 사용전 미리 읽어보면 누구나 쉽게 사용할 수 있을 것